In [11]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.compat.v2.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

In [3]:
train_data = pd.read_csv("../train_data.csv")
dev_data = pd.read_csv("../dev_data.csv")

train_data = train_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
train_data["is_retweet"] = 1*train_data["is_retweet"]
train_data["is_modified_retweet"] = 1*train_data["is_modified_retweet"]

dev_data = dev_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
dev_data["is_retweet"] = 1*dev_data["is_retweet"]
dev_data["is_modified_retweet"] = 1*dev_data["is_modified_retweet"]

In [4]:
# X_train = train_data.drop(['tweet','author_Id'],axis=1).values
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# y_train = to_categorical(train_data["author_Id"])

sc = train_data.groupby("author_Id").count().sort_values("tweet",ascending = False)#.head()
set_y = list(sc.index)

In [6]:
def id_to_int_cat(id_ind):
    return np.argwhere(train_data["author_Id"].values[id_ind]==set_y)[0][0]
temp = [id_to_int_cat(i) for i in range(len(train_data["author_Id"]))]
y_train = to_categorical(temp,dtype=int)

In [7]:
X_train = train_data.drop(['tweet','author_Id'],axis=1).values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = to_categorical(temp,dtype=int)

In [8]:
def make_ff():
    model = Sequential()
    model.add(Dense(1000,activation = "relu",input_dim=X_train.shape[1],kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # model.add(Dense(5000,activation = "relu",kernel_regularizer=regularizers.l2(0.01)))
    # model.add(Dropout(0.3))
    model.add(Dense(y_train.shape[1],activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'],)
    
    return model


In [12]:
filepath = "modeln.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop =  EarlyStopping(monitor='val_loss',patience=3, min_delta=0.0001)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logbn")
callbacks_list = [checkpoint,tensorboard_callback]#,early_stop,]


epochs = 3
batch_size = 128
model = make_ff()
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.25,
                    callbacks=callbacks_list)

W0906 19:01:50.897455 4636804544 deprecation.py:323] From /anaconda3/envs/TF/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 185024 samples, validate on 61675 samples
Epoch 1/3
184960/185024 [============================>.] - ETA: 0s - loss: 8.7479 - accuracy: 0.0043
Epoch 00001: val_loss improved from inf to 8.53470, saving model to modeln.h5
185024/185024 [==============================] - 226s 1ms/sample - loss: 8.7478 - accuracy: 0.0043 - val_loss: 8.5347 - val_accuracy: 0.0077
Epoch 2/3
184960/185024 [============================>.] - ETA: 0s - loss: 8.3803 - accuracy: 0.0082
Epoch 00002: val_loss improved from 8.53470 to 8.37557, saving model to modeln.h5
185024/185024 [==============================] - 213s 1ms/sample - loss: 8.3803 - accuracy: 0.0082 - val_loss: 8.3756 - val_accuracy: 0.0100
Epoch 3/3
184960/185024 [============================>.] - ETA: 0s - loss: 8.2116 - accuracy: 0.0105
Epoch 00003: val_loss improved from 8.37557 to 8.31296, saving model to modeln.h5
185024/185024 [==============================] - 245s 1ms/sample - loss: 8.2116 - accuracy: 0.0105 - val_loss: 8.3130 - va

In [ ]:
%tensorboard --logdir log

In [45]:
!pwd

/Users/maharaj/Documents/WhoTweeted/split_model


In [ ]:
new_model = load_model("modeln.h5")
epochs = 100
batch_size = 64

history = new_model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.25,
                    callbacks=callbacks_list)

Train on 185024 samples, validate on 61675 samples
Epoch 1/100
184960/185024 [============================>.] - ETA: 0s - loss: 8.7802 - accuracy: 0.0024
Epoch 00001: val_loss improved from 8.84868 to 8.79987, saving model to model.h5
185024/185024 [==============================] - 66s 356us/sample - loss: 8.7802 - accuracy: 0.0024 - val_loss: 8.7999 - val_accuracy: 0.0028
Epoch 2/100
184832/185024 [============================>.] - ETA: 0s - loss: 8.7224 - accuracy: 0.0028
Epoch 00002: val_loss improved from 8.79987 to 8.76508, saving model to model.h5
185024/185024 [==============================] - 61s 328us/sample - loss: 8.7224 - accuracy: 0.0028 - val_loss: 8.7651 - val_accuracy: 0.0030
Epoch 3/100
184832/185024 [============================>.] - ETA: 0s - loss: 8.6822 - accuracy: 0.0031
Epoch 00003: val_loss improved from 8.76508 to 8.73986, saving model to model.h5
185024/185024 [==============================] - 59s 319us/sample - loss: 8.6823 - accuracy: 0.0031 - val_loss: 8

Epoch 26/100
184832/185024 [============================>.] - ETA: 0s - loss: 8.4590 - accuracy: 0.0052
Epoch 00026: val_loss did not improve from 8.60812
185024/185024 [==============================] - 57s 307us/sample - loss: 8.4592 - accuracy: 0.0052 - val_loss: 8.6101 - val_accuracy: 0.0054
Epoch 27/100
184960/185024 [============================>.] - ETA: 0s - loss: 8.4546 - accuracy: 0.0051
Epoch 00027: val_loss improved from 8.60812 to 8.60808, saving model to model.h5
185024/185024 [==============================] - 56s 302us/sample - loss: 8.4546 - accuracy: 0.0051 - val_loss: 8.6081 - val_accuracy: 0.0051
Epoch 28/100
184896/185024 [============================>.] - ETA: 0s - loss: 8.4496 - accuracy: 0.0051
Epoch 00028: val_loss improved from 8.60808 to 8.60384, saving model to model.h5
185024/185024 [==============================] - 57s 306us/sample - loss: 8.4497 - accuracy: 0.0051 - val_loss: 8.6038 - val_accuracy: 0.0057
Epoch 29/100
184832/185024 [=====================

Epoch 52/100
184896/185024 [============================>.] - ETA: 0s - loss: 8.3585 - accuracy: 0.0065
Epoch 00052: val_loss improved from 8.55262 to 8.55172, saving model to model.h5
185024/185024 [==============================] - 83s 447us/sample - loss: 8.3585 - accuracy: 0.0065 - val_loss: 8.5517 - val_accuracy: 0.0067
Epoch 53/100
184960/185024 [============================>.] - ETA: 0s - loss: 8.3564 - accuracy: 0.0067